In [40]:
# Import packages
import pandas as pd

# File paths
input_path = '../data/'
output_path = '../data/combined/'

production_file = input_path+'extracted/IFA_production_w_uncertainties.csv'
conversion_factor_file = input_path+'extracted/conversionFactors_from_IFA_w_uncertainties.csv'

In [41]:
production = pd.read_csv(production_file)
conversions = pd.read_csv(conversion_factor_file, index_col=0)

In [42]:
name_conversions = {
    'NH3': 'AMMONIA',
    'AN': 'AMMONIUM NITRATE',
    'Ammonium nitrate (33.5-0-0) granulated': 'AMMONIUM NITRATE',
    'AS': 'AMMONIUM SULPHATE',
    'CAN': 'CALCIUM AMMONIUM NITRATE',
    'Calcium ammonium nitrate (27-0-0)': 'CALCIUM AMMONIUM NITRATE',
    'Urea (46-0-0)': 'UREA'
}

production['PRODUCT'] = production['PRODUCT'].replace(name_conversions)
conversions['Product'] = conversions['Product'].replace(name_conversions).str.upper()

In [43]:
prod_conv = pd.merge(production, conversions, how='left', left_on='PRODUCT', right_on='Product')

names = ['CtOG', 'Raw Material', 'Indirect Utilities', 'Direct Utilities', 'Direct Process']

years = [str(i) for i in list(range(2009, 2021))]
years_sigma = [str(year)+'_sigma' for year in years]

base_columns = ['PRODUCT', 'Region'] + years + years_sigma
base_df = production[base_columns]

calc_emissions = pd.DataFrame()
db, gas = 'ifa', 'CO2e_100a'
for name in names:
    df = base_df.copy()
    df[years] = df[years].multiply(prod_conv[db+'_'+gas+'_'+name+' factor'], axis='index')
    ## Incorrect error propagation here
    df[years_sigma] = df[years_sigma].multiply(prod_conv[db+'_'+gas+'_'+name+' sigma'], axis='index')
    df['Gas'] = gas
    df['Type'] = name
    calc_emissions = pd.concat((calc_emissions, df), axis = 0)

In [44]:
calc_emissions.to_csv(output_path+'ifaProductionEmissions.csv')